# Compare Cloud-Optimized Geotiffs from ASDI "sentinel-s1-rtc-indigo" S3 buckets 

This notebook shows an option on how to construct a multidimensional xarray DataArray from multiple 2D Images COGS

# Sentinel-1 Radiometric Terrain Corrected (RTC) Data

https://sentinel-s1-rtc-indigo-docs.s3-us-west-2.amazonaws.com/index.html

The Sentinel-1 mission is a constellation of two C-band Synthetic Aperature Radar (SAR) imaging satellites from the European Space Agency (ESA). These satellites were launched in 2014 (S1A) and 2016 (S1B) and are capable of providing imagery in all-weather, day or night, under a few different observation configurations. Each satellite has a repeat cycle of 12-days and completes 175 orbits per cycle, giving it a potential 6-day repeat coverage time.

 he Sentinel-1 data included in this dataset are from the “Interferometric Wide Swath” (IW) mode, which is the main acquisition mode over land. The data were first preprocesed by ESA into a Level-1 “Ground Range Detected” (GRD) product, which involves detection, multi-looking, and projection to ground-range using a simple Earth ellipsoid model.

We have applied additional processing steps on top of the “GRD” product to make it “Radiometric Terrain Corrected” (RTC), including calibration, noise removal, speckle filtering, and radiometric and terrain correction. We also transform the dataset from a collection of “scenes”, or slices of orbital passes, into a regularly gridded, “tiled” data product. We perform these steps to make the imagery suitable for analysis, but some preprocessing choices may not fit your application. For more information, see the Methodology section.

This product is available for the Contiguous United States (CONUS) only, which has been mapped routinely over the enter domain since 2017 in dual-vertical (DV) polarization mode on ascending orbits. We update the dataset with newly acquired Sentinel-1 data on a daily cadence.

In [75]:
# please use the environement.ymal file to create the needed environement with conda
#%pip install rioxarray intake intake-xarray rasterio datashader s3fs hvplot

In [76]:
import hvplot
import dask
import s3fs
import intake
import os
import rasterio
import rioxarray
import xarray as xr
import pandas as pd

In [77]:
# use the same GDAL environment settings as we did for the single COG case
env = dict(GDAL_DISABLE_READDIR_ON_OPEN='EMPTY_DIR', 
           AWS_NO_SIGN_REQUEST='YES',
           GDAL_MAX_RAW_BLOCK_CACHE_SIZE='2000000000',
           GDAL_SWATH_SIZE='2000000000',
           VSI_CURL_CACHE_SIZE='2000000000')
os.environ.update(env)

#define year in scope
year = '2020'


https://sentinel-s1-rtc-indigo-docs.s3-us-west-2.amazonaws.com/data_format.html

In [78]:
# set up a connection with credentials and other settings
s3 = s3fs.S3FileSystem(anon=True)
objects = s3.ls('sentinel-s1-rtc-indigo/tiles/RTC/1/IW/19/T/DK/' + year + '/')
images = ['s3://' + obj + '/Gamma0_VV.tif' for obj in objects]
print(len(images))
images[:] 

#aws s3 ls --no-sign-request s3://sentinel-s1-rtc-indigo/tiles/RTC/1/IW/10/T/ET/
#s3://sentinel-s1-rtc-indigo/tiles/RTC/1/[MODE]=IW/[MGRS UTM zone]/[MGRS latitude label]/[MGRS Grid Square ID]/[YEAR]/[SATELLITE]_[DATE]_[TILE ID]_[ORBIT DIRECTION]/[ASSET]

79


['s3://sentinel-s1-rtc-indigo/tiles/RTC/1/IW/19/T/DK/2020/S1A_20200106_19TDK_ASC/Gamma0_VV.tif',
 's3://sentinel-s1-rtc-indigo/tiles/RTC/1/IW/19/T/DK/2020/S1A_20200111_19TDK_ASC/Gamma0_VV.tif',
 's3://sentinel-s1-rtc-indigo/tiles/RTC/1/IW/19/T/DK/2020/S1A_20200118_19TDK_ASC/Gamma0_VV.tif',
 's3://sentinel-s1-rtc-indigo/tiles/RTC/1/IW/19/T/DK/2020/S1A_20200123_19TDK_ASC/Gamma0_VV.tif',
 's3://sentinel-s1-rtc-indigo/tiles/RTC/1/IW/19/T/DK/2020/S1A_20200130_19TDK_ASC/Gamma0_VV.tif',
 's3://sentinel-s1-rtc-indigo/tiles/RTC/1/IW/19/T/DK/2020/S1A_20200204_19TDK_ASC/Gamma0_VV.tif',
 's3://sentinel-s1-rtc-indigo/tiles/RTC/1/IW/19/T/DK/2020/S1A_20200211_19TDK_ASC/Gamma0_VV.tif',
 's3://sentinel-s1-rtc-indigo/tiles/RTC/1/IW/19/T/DK/2020/S1A_20200216_19TDK_ASC/Gamma0_VV.tif',
 's3://sentinel-s1-rtc-indigo/tiles/RTC/1/IW/19/T/DK/2020/S1A_20200223_19TDK_ASC/Gamma0_VV.tif',
 's3://sentinel-s1-rtc-indigo/tiles/RTC/1/IW/19/T/DK/2020/S1A_20200228_19TDK_ASC/Gamma0_VV.tif',
 's3://sentinel-s1-rtc-indigo/

A GDAL VRT file is an XML format that can group together many separate files into separate bands. 
It's common to create such a file with a the GDAL command line tool `gdalbuildvrt`, illustrated below:

In [79]:
#step 1) write a file list that points to the data. GDAL requires special prefixes for this /vsis3/ or /vsicurl/
with open('files_runtime_inventory.txt', 'w') as f:
    lines = [x.replace('s3://', '/vsis3/') + '\n' for x in images[:]]
    f.writelines(lines)
#print(lines)

In [80]:
%%time
# step 2) create a VRT file
!gdalbuildvrt stack.vrt -separate -input_file_list files_runtime_inventory.txt 

0...10...20...30...40...50...60...70...80...90...100 - done.
CPU times: user 751 ms, sys: 181 ms, total: 933 ms
Wall time: 31.2 s


In [90]:
%%time
# step 3) open with xarray
chunks=dict(band=1, x=2745, y=2745)
#da = xr.open_rasterio('stack.vrt', chunks=chunks)
ds = xr.open_dataset('stack.vrt', chunks=chunks, engine="rasterio")
ds

CPU times: user 10 ms, sys: 4.34 ms, total: 14.4 ms
Wall time: 18.4 ms


<xarray.Dataset>
Dimensions:      (band: 79, x: 5490, y: 5490)
Coordinates:
  * band         (band) int64 1 2 3 4 5 6 7 8 9 ... 71 72 73 74 75 76 77 78 79
  * x            (x) float64 4e+05 4e+05 4e+05 ... 5.097e+05 5.097e+05 5.098e+05
  * y            (y) float64 5e+06 5e+06 5e+06 ... 4.89e+06 4.89e+06 4.89e+06
    spatial_ref  int64 ...
Data variables:
    band_data    (band, y, x) float32 dask.array<chunksize=(1, 2745, 2745), meta=np.ndarray>

In [91]:
# step 4) optionally modify coordinates (e.g. time dimension extracted from file name)
#ds = ds.rename({'band':'time'})
ds['time'] = [pd.to_datetime(x[60:68]) for x in images[:]]

1. `xr.open_rasterio(stack.vrt)` stores band coordinates as sequential integers (we lose file name and metadata from each individual COG, so it's common to alter the coordinates after opening the dataset)
2. data is tied to a reference to a local file ('stack.vrt'), which can cause problems with distributed computing if you don't have access to the local filesystem

## intake-xarray

[intake-xarray](https://github.com/intake/intake-xarray) is a plugin for the intake library. It uses fsspec/s3fs under the hood to facilitate loading data into python objects. the function `intake.open_rasterio()` accepts a list of paths. it returns an intake object with a `to_dask()` function that returns an xarray DataArray

In [92]:
%%time
# ~ 1s for 6 files

# this loads the image ID into xarray's band coordinates. 

pattern = 's3://sentinel-s1-rtc-indigo/tiles/RTC/1/IW/19/T/DK/' + year + '/{band}/Gamma0_VV.tif'
chunks=dict(band=1, x=5490, y=5490)
sources = intake.open_rasterio(images[:], chunks=chunks, path_as_pattern=pattern, concat_dim='band')
ds = sources.to_dask() 
ds

CPU times: user 228 ms, sys: 5.66 ms, total: 233 ms
Wall time: 234 ms


<xarray.DataArray (band: 79, y: 5490, x: 5490)>
dask.array<concatenate, shape=(79, 5490, 5490), dtype=float32, chunksize=(1, 5490, 5490), chunktype=numpy.ndarray>
Coordinates:
  * band     (band) <U22 'S1A_20200106_19TDK_ASC' ... 'S1B_20201229_19TDK_DSC'
  * y        (y) float64 5e+06 5e+06 5e+06 5e+06 ... 4.89e+06 4.89e+06 4.89e+06
  * x        (x) float64 4e+05 4e+05 4e+05 ... 5.097e+05 5.097e+05 5.098e+05
Attributes:
    transform:              (20.0, 0.0, 399960.0, 0.0, -20.0, 5000040.0)
    crs:                    +init=epsg:32619
    res:                    (20.0, 20.0)
    is_tiled:               1
    nodatavals:             (0.0,)
    scales:                 (1.0,)
    offsets:                (0.0,)
    descriptions:           ('Gamma0_VV',)
    ABSOLUTE_ORBIT_NUMBER:  30686
    AREA_OR_POINT:          Area
    DATE:                   2020-01-06
    MISSION_ID:             S1A
    NUMBER_SCENES:          1
    ORBIT_DIRECTION:        ascending
    OVR_RESAMPLING_ALG:     AVERAGE
    SCENES:                 S1A_IW_GRDH_1SDV_20200106T222728_20200106T222753_...
    SCENE_1_METADATA:       {"title": "S1A_IW_GRDH_1SDV_20200106T222728_20200...
    SCENE_1_PRODUCT_INFO:   {"id": "S1A_IW_GRDH_1SDV_20200106T222728_20200106...
    TILE_ID:                19TDK
    VALID_PIXEL_PERCENT:    86.537

In [93]:
import hvplot.xarray
ds.hvplot.image(rasterize=True, aspect='equal', cmap='gray', clim=(0,0.4), frame_width=600, frame_height=600)

:DynamicMap   [band]
   :Image   [x,y]   (value)

## Custom

You can also just use xarray and dask to construct a larger datacube from many COGS. 

In [ ]:
%%time

# 4 - 8 s
# Load all the images

chunks=dict(band=1, x=2745, y=2745)
dataArrays = [xr.open_dataset(url, chunks=chunks, engine="rasterio") for url in images]

# note use of join='override' b/c we know these COGS have the same coordinates
da = xr.concat(dataArrays, dim='band', join='override', combine_attrs='drop')
da = da.rename({'band':'time'})
da['time'] = [pd.to_datetime(x[60:68]) for x in images]
da

#### recap:

* The cell above is essentially a for-loop that iterates over each COG in sequence. 50ms-200ms * 80 ~ 4-16 seconds. The next notebook will look at using Dask to speed things up by opening the files in parallel.

## Visualize

Here is an example of interactive visualization again using hvplot. Since we're using full resolution arrays it's key to set the `rasterize=True` keyword argument. That uses the datashader library to pre-render images before sending them to the browser.

This is extremely powerful because, resolution updates as you zoom in, and you can scrub through the data cube with an interactive slider widget

In [ ]:
import hvplot.xarray
da.hvplot.image(rasterize=True, aspect='equal', cmap='gray', clim=(0,0.4))